In [1]:

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [2]:
import os
import numpy as np
import glob|
import csv
import keras
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten


from inception_model_generator import MRNet_inc_model
import tensorflow as tf

from numpy.random import seed
seed(1)
tf.random.set_seed(2)

SyntaxError: ignored

In [5]:

b_size = 1

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)
data_gen = MRNet_data_generator(data_path, p, l, batch_size=b_size, scale_to=(299,299), model="inception")

model, cp_callback = MRNet_inc_model(["abnormal", "axial"])


# print(x.shape)
model.fit(x=data_gen, epochs=3, verbose=True, callbacks=[cp_callback], shuffle=False, use_multiprocessing=True, workers=6)
# model.save_weights("training_inc_1/w.ckpt")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 2)                 23247858  
Total params: 23,247,858
Trainable params: 23,209,554
Non-trainable params: 38,304
_________________________________________________________________


KeyboardInterrupt: ignored

In [0]:
checkpoint_path = "training_inc_1/cp.ckpt"

! mkdir "./training_inc_1/"

In [0]:
b_size = 1

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)
data_gen = MRNet_data_generator(data_path, p, l, batch_size=b_size, scale_to=(299,299), model="inception")

model = MRNet_inc_model(["abnormal", "axial"])


# print(x.shape)
model.fit(x=data_gen, epochs=3, verbose=True, callbacks=[cp_callback], shuffle=False, use_multiprocessing=True, workers=6)
# model.save_weights("training_inc_1/w.ckpt")

In [0]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   .gitignore
	modified:   __pycache__/data_generator.cpython-36.pyc
	modified:   __pycache__/inception_model_generator.cpython-36.pyc
	modified:   __pycache__/partition.cpython-36.pyc
	new file:   dataset/MRNet-v1.0/valid/sagittal/.gitignore
	modified:   inception_functional.ipynb
	modified:   inception_model_generator.py



In [0]:
from google.colab import drive
drive.mount('/content/drive')